In [3]:
import numpy as np

In [7]:
np.loadtxt('..\\data\\ml-100k\\ua.test', skiprows=0, delimiter='\t').astype('int32')

array([[        1,        20,         4, 887431883],
       [        1,        33,         4, 878542699],
       [        1,        61,         4, 878542420],
       ...,
       [      943,       570,         1, 888640125],
       [      943,       808,         4, 888639868],
       [      943,      1067,         2, 875501756]])

In [19]:
def load_data(path, delimiter='\t'):
    train = np.loadtxt(path+'ua.base', skiprows=0, delimiter=delimiter).astype('int32')
    test = np.loadtxt(path+'ua.test', skiprows=0, delimiter=delimiter).astype('int32')
    total = np.concatenate((train, test), axis=0)

    n_u = np.unique(total[:, 0]).size #num of users
    n_i = np.unique(total[:, 1]).size #num of items

    train_data = np.zeros((n_u, n_i), dtype='float32')
    test_data = np.zeros((n_u, n_i), dtype='float32')

    for i in range(train.shape[0]):
        train_data[train[i][0]-1][train[i][1]-1] = train[i][2]
    
    for i in range(test.shape[0]):
        test_data[test[i][0]-1][test[i][1]-1] = test[i][2]

    return train_data, test_data

In [20]:
train_data, test_data = load_data(path='..\\data\\ml-100k\\')

In [23]:
train_data.shape

(943, 1682)

In [ ]:
def average_item_rating(train_data):
    